# Setup/Test Environment:

In [1]:
from web_footprinting_utils import setup_ipython
import os

get_ipython, INTERACTIVE = setup_ipython()

In [2]:
%env TARGET=10.10.220.12

env: TARGET=10.10.220.12


In [3]:
!echo Make sure we are connected to VPN:
result = !ip a | grep tun0

if not len(result)>0:
    print("Not connected!")
    input()
else:
    print("Connected!")
    print(result)

Make sure we are connected to VPN:
Connected!
['6: tun0: <POINTOPOINT,MULTICAST,NOARP,UP,LOWER_UP> mtu 1500 qdisc pfifo_fast state UNKNOWN group default qlen 500', '    inet 10.2.91.54/17 scope global tun0']


# Initial (Unreliable) RustScan:

In [4]:
import os
pwd = os.getcwd()
rustscan = "docker run -v '%s':/workspace  --volume /etc/passwd:/etc/passwd:ro --volume /etc/group:/etc/group:ro --user $(id -u) --rm dnola/rustscan" % pwd

In [18]:
!{rustscan} -b 1000 -t 4000 -r 1-65535 -a {os.environ["TARGET"]} -- -Pn -A -sV

.----. .-. .-. .----..---.  .----. .---.   .--.  .-. .-.
| {}  }| { } |{ {__ {_   _}{ {__  /  ___} / {} \ |  `| |
| .-. \| {_} |.-._} } | |  .-._} }\     }/  /\  \| |\  |
`-' `-'`-----'`----'  `-'  `----'  `---' `-'  `-'`-' `-'
The Modern Day Port Scanner.
________________________________________
: https://discord.gg/GFrQsGy           :
: https://github.com/RustScan/RustScan :
 --------------------------------------
Please contribute more quotes to our GitHub https://github.com/rustscan/rustscan

[~] The config file is expected to be at "/home/parallels/.rustscan.toml"
[~] File limit higher than batch size. Can increase speed by increasing batch size '-b 1048476'.
Open 10.10.220.12:21
Open 10.10.220.12:3389
Open 10.10.220.12:9999
[~] Starting Script(s)
[>] Running script "nmap -vvv -p {{port}} {{ip}} -Pn -A -sV" on ip 10.10.220.12
Depending on the complexity of the script, results may take some time to appear.



KeyboardInterrupt



# Initial Nmap:

In [19]:
!echo $TARGET
!mkdir -p scans/quick scans/tcp scans/udp scans/scripts scans/http scans/no_ping

10.10.220.12


In [20]:
!ping -c 5 $TARGET

PING 10.10.220.12 (10.10.220.12) 56(84) bytes of data.

--- 10.10.220.12 ping statistics ---
3 packets transmitted, 0 received, 100% packet loss, time 2049ms




KeyboardInterrupt



# Quick Scan:

In [21]:
!nmap -sV -vv -oA scans/quick/quick_scan $TARGET

Starting Nmap 7.91 ( https://nmap.org ) at 2021-09-26 08:35 PDT
NSE: Loaded 45 scripts for scanning.
Initiating Ping Scan at 08:35
Scanning 10.10.220.12 [2 ports]
Completed Ping Scan at 08:35, 3.01s elapsed (1 total hosts)
Read data files from: /usr/bin/../share/nmap
Note: Host seems down. If it is really up, but blocking our ping probes, try -Pn
Nmap done: 1 IP address (0 hosts up) scanned in 3.15 seconds


# All Ports:

In [22]:
!nmap -p- -sV -vv -oA  scans/tcp/tcp_scan $TARGET

Starting Nmap 7.91 ( https://nmap.org ) at 2021-09-26 08:35 PDT
NSE: Loaded 45 scripts for scanning.
Initiating Ping Scan at 08:35
Scanning 10.10.220.12 [2 ports]
Completed Ping Scan at 08:35, 3.00s elapsed (1 total hosts)
Read data files from: /usr/bin/../share/nmap
Note: Host seems down. If it is really up, but blocking our ping probes, try -Pn
Nmap done: 1 IP address (0 hosts up) scanned in 3.13 seconds


In [23]:
import xmltodict

with open('scans/tcp/tcp_scan.xml','r') as f:
    xml_dict = xmltodict.parse(f.read())

In [24]:
PING=True
try:
    port_list = [port['@portid'] for port in xml_dict['nmaprun']['host']['ports']['port']]
    %env PORTS={",".join(port_list)}
except:
    print("Host is down")
    PING=False

Host is down


# No Ping:

In [32]:
!nmap -sV -Pn -vv -oA scans/no_ping/no_ping_scan $TARGET

Host discovery disabled (-Pn). All addresses will be marked 'up' and scan times will be slower.


Starting Nmap 7.91 ( https://nmap.org ) at 2021-09-26 08:36 PDT
NSE: Loaded 45 scripts for scanning.
Initiating Parallel DNS resolution of 1 host. at 08:36
Completed Parallel DNS resolution of 1 host. at 08:36, 0.01s elapsed
Initiating Connect Scan at 08:36
Scanning 10.10.220.12 [1000 ports]
Discovered open port 3389/tcp on 10.10.220.12
Discovered open port 21/tcp on 10.10.220.12
Discovered open port 9999/tcp on 10.10.220.12
Completed Connect Scan at 08:37, 29.95s elapsed (1000 total ports)
Initiating Service scan at 08:37



KeyboardInterrupt



In [33]:
if not PING:
    !nmap -sV -Pn -p- -vv -oA  scans/tcp/tcp_scan $TARGET

Host discovery disabled (-Pn). All addresses will be marked 'up' and scan times will be slower.


Starting Nmap 7.91 ( https://nmap.org ) at 2021-09-26 08:37 PDT
NSE: Loaded 45 scripts for scanning.
Initiating Parallel DNS resolution of 1 host. at 08:37
Completed Parallel DNS resolution of 1 host. at 08:37, 0.01s elapsed
Initiating Connect Scan at 08:37
Scanning 10.10.220.12 [65535 ports]
Discovered open port 3389/tcp on 10.10.220.12
Discovered open port 21/tcp on 10.10.220.12
Connect Scan Timing: About 7.38% done; ETC: 08:44 (0:06:29 remaining)
Connect Scan Timing: About 19.50% done; ETC: 08:42 (0:04:12 remaining)
Connect Scan Timing: About 35.27% done; ETC: 08:41 (0:02:47 remaining)
Connect Scan Timing: About 35.87% done; ETC: 08:43 (0:03:36 remaining)
Increasing send delay for 10.10.220.12 from 0 to 5 due to 26 out of 86 dropped probes since last increase.
Connect Scan Timing: About 36.00% done; ETC: 08:44 (0:04:28 remaining)
Increasing send delay for 10.10.220.12 from 5 to 10 due to 11 out of 11 dropped probes since last increase.
Increasing send delay for 10.10.220.12 from 10 


KeyboardInterrupt



# Run Default Scripts:

In [27]:
if PING:
    !nmap -sC -p $PORTS -vv -oA scans/scripts/script_scan $TARGET

In [28]:
if PING:
    !nmap -A -T5 -vv --script=default,safe,discovery $TARGET
else:
    !nmap -A -T5 -Pn -vv --script=default,safe,discovery $TARGET

Host discovery disabled (-Pn). All addresses will be marked 'up' and scan times will be slower.


Starting Nmap 7.91 ( https://nmap.org ) at 2021-09-26 08:35 PDT



KeyboardInterrupt



In [29]:
if PING:
    !nmap -A -T3 -vv --script=default,safe,discovery $TARGET
else:
    !nmap -A -T3 -Pn -vv --script=default,safe,discovery $TARGET

Host discovery disabled (-Pn). All addresses will be marked 'up' and scan times will be slower.


Starting Nmap 7.91 ( https://nmap.org ) at 2021-09-26 08:35 PDT



KeyboardInterrupt



In [30]:
from getpass import getpass
import os

if PING:
    !echo { getpass() } | sudo -S nmap -A -p- -T5 -vv --script=default,safe,discovery {os.environ['TARGET']}
else:
    !echo { getpass() } | sudo -S nmap -A -p- -Pn -T5 -vv --script=default,safe,discovery {os.environ['TARGET']}

KeyboardInterrupt: Interrupted by user

In [ ]:
# UDP Scan, this takes forever...

# from getpass import getpass

# !echo { getpass() } > tmp.txt
# !cat tmp.txt | sudo -S nmap -sU --open -oA scans/udp/udp_scan $TARGET
# !cat tmp.txt | sudo -S nmap -sU --open -p- -oA scans/udp/udp_scan $TARGET
# !rm -rf tmp.txt